# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Clone `CreateDebateScraper` library from github
!git clone https://github.com/utkarsh512/CreateDebateScraper.git
%cd CreateDebateScraper/src/nested/

In [ ]:
from   copy                     import deepcopy
from   itertools                import accumulate
import json
from   matplotlib               import pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import pickle
import re
from   scipy                    import stats
import textwrap
from   thread                   import Comment, Thread
from   tqdm                     import tqdm
nltk.download('punkt') # For tokenizers
nltk.download('stopwords')
import matplotlib
from   nltk.tokenize            import TweetTokenizer
from   nltk.corpus              import stopwords
from   pprint                   import pprint
import wordcloud
# import skbio
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams["figure.figsize"] = (12, 5)
STOP_WORDS = list(stopwords.words('english'))

# Helper routines

In [ ]:
tknz = TweetTokenizer()

def clean_text(text):
    """
    Preprocessing text
    """
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub("-", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\u2018", "X", text) 
    text = re.sub("\u2019", "X", text) 
    text = re.sub("\'", "X", text) 
    wordTokens_ = tknz.tokenize(text)
    wordTokens = list()
    for x in wordTokens_:
        x = ''.join([v for v in x if v.isalnum() or v == ' '])
        if len(x) > 0 and x != 'X':
            x = x.replace('X', '\'')
            wordTokens.append(x)
    return wordTokens

# Load CreateDebate dataset

In [ ]:
comments = dict()

# Topical forums on CreateDebate. We have scraped comments for all of the
# following forurm.
categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

# However, we will be analyzing comments from selected forum only!
# These forum have at least 10k comments each.
categories_selected = ['politics2', 'religion', 'world', 
                       'science', 'law', 'technology']

for x in categories_selected:
    comments[x] = list()

In [ ]:
# Loading comments from select forums

for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')

    # Get all the `Thread` objects pickled while scraping.
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()

    # While classifying CreateDebate comments, we used comments as per author mode.
    # Hence, using the same mode to attach classification score with the comments.
    # 
    # score < 0.5 -> ad hominem comment
    #       > 0.5 -> non ad hominem comment
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append(v)
            except:
                authors[v.author] = list()
                authors[v.author].append(v)

    ctr = 0
    # Load the classification score of the comments.
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    # Attach classification score with the comments.
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            foo['score'] = cws[ctr][0]
            foo['validation'] = cws[ctr][1][0]
            comments[cat].append(foo)
            ctr += 1

# Load AH score

In [ ]:
# Loading computation from cache

with open('/content/gdrive/MyDrive/Temp/47-ah-score.pkl', 'rb') as fp:
    ah_score = pickle.load(fp)

# Load CreateDebate profile dataset

In [ ]:
df = pd.read_json('/content/gdrive/MyDrive/DL/CreateDebate/profile/results.json', lines=True)

In [ ]:
reward_points_map = {k : v for k, v in zip(df['username'].tolist(), df['reward_points'].tolist())}
efficiency_map    = {k : v for k, v in zip(df['username'].tolist(), df['efficiency'].tolist())}
allies_map        = {k : len(v) for k, v in zip(df['username'].tolist(), df['allies'].tolist())}
enemies_map       = {k : len(v) for k, v in zip(df['username'].tolist(), df['enemies'].tolist())}
hostiles_map      = {k : len(v) for k, v in zip(df['username'].tolist(), df['hostiles'].tolist())}

In [ ]:
def get_stats_from_profile_data(user_subset):
    rewards_ = list()
    efficiency_ = list()
    n_allies = list()
    n_enemies = list()
    n_hostiles = list()

    for user in user_subset:
        rewards_.append(reward_points_map[user])
        efficiency_.append(efficiency_map[user])
        n_allies.append(allies_map[user])
        n_enemies.append(enemies_map[user])
        n_hostiles.append(hostiles_map[user])
    
    grpd_data = [rewards_, efficiency_, n_allies, n_enemies, n_hostiles]
    avgs = [np.average(x) for x in grpd_data]
    stds = [np.std(x) for x in grpd_data]
    
    return avgs, stds

# Computing `ah_score_median`, `comment_count`, `comment_count_user` and `user_list`

In [ ]:
# Median ah score per category per author
#   key: category -> author
#   value: median ah score
ah_score_median = dict()

for category, author_data in ah_score.items():
    ah_score_median[category] = dict()
    for author, ah_scores in author_data.items():
        ah_score_median[category][author] = np.median(ah_scores)

In [ ]:
comment_count = dict()
# key: category -> author
# value: number of comments written by author in the given forum

for category in categories_selected:
    comment_count[category] = dict()

    for comment in comments[category]:
        author = comment['author']
        try:
            comment_count[category][author] += 1
        except KeyError:
            comment_count[category][author] = 1

In [ ]:
user_list = set()

for category in categories_selected:
    for comment in comments[category]:
        user_list.add(comment['author'])

user_list = list(user_list)

In [ ]:
comment_count_user = dict()
# key: user -> category -> year (in string)
# value: number of comments posted by the user for that category in the given year

for user in user_list:
    comment_count_user[user] = dict()
    for category in categories_selected:
        comment_count_user[user][category] = dict()
        for year in range(2008, 2022):
            syear = str(year)
            comment_count_user[user][category][syear] = 0


for category in categories_selected:
    for comment in comments[category]:
        if comment['time'] == 'Not Available':
            continue
        year = comment['time'][:4]
        assert(int(year) < 2022 and int(year) >= 2008)
        comment_count_user[ comment['author'] ][ category ][ year ] += 1 

# User migration

In [ ]:
def get_migrated_users(categories_1, categories_2):
    """
    Returns a list of users who were active on forums included in `categories_1`,
    but later started posting in forums included in `categories_2`
    """
    if not (isinstance(categories_1, list) and isinstance(categories_2, list)):
        raise ValueError('Arguments should be list')

    user_yom_list = list()
    for user in user_list:
        count_1 = [0 for _ in range(2008, 2022)]
        count_2 = [0 for _ in range(2008, 2022)]

        for year in range(2008, 2022):
            for category in categories_1:
                count_1[year - 2008] += comment_count_user[user][category][str(year)]
            for category in categories_2:
                count_2[year - 2008] += comment_count_user[user][category][str(year)] 

        idx_nz_cat1 = -1   # index of first non-zero entry in count_1
        idx_nz_cat2 = -1   # index of first non-zero entry in count_2

        for i in range(len(count_1)):
            if count_1[i] != 0:
                idx_nz_cat1 = i
                break
            
        for i in range(len(count_2)):
            if count_2[i] != 0:
                idx_nz_cat2 = i
                break
        
        if idx_nz_cat1 == -1 or idx_nz_cat2 == -1:
            # This user has not posted any comment in forums included in 
            # categories_1 / categories_2. Hence, go to next user.
            continue

        if idx_nz_cat1 >= idx_nz_cat2:
            # User first posted in forum for cat2, ignore this user
            continue

        # Check if this user has at least one comment per user in count_1 till
        # he migrated to other forums
        ok = True
        for i in range(idx_nz_cat1, idx_nz_cat2):
            ok = ok and (count_1[i] != 0)
        
        # Append (user, year_of_migration) pair
        user_yom_list.append((user, (idx_nz_cat1 + 2008, idx_nz_cat2 + 2008)))

    return user_yom_list

In [ ]:
# Partition the result of above function into extreme, moderate and low AH groups

EXTREME_AH_UPPER = 1
EXTREME_AH_LOWER = 0.95

MODERATE_AH_UPPER = 0.8
MODERATE_AH_LOWER = 0.7

LOW_AH_UPPER = 0.6
LOW_AH_LOWER = 0.5


def partition_migrated_users(user_yom_list, categories_1, categories_2):
    extreme_user_yom_list = list()
    moderate_user_yom_list = list()
    low_user_yom_list = list()

    for user, yom in user_yom_list:
        user_ah_score = []
        for category in (categories_1 + categories_2):
            try:
                user_ah_score.append(ah_score_median[category][user])
            except:
                pass
        
        user_ah_score = np.average(user_ah_score)

        if user_ah_score <= EXTREME_AH_UPPER and user_ah_score >= EXTREME_AH_LOWER:
            extreme_user_yom_list.append((user, yom))
        
        elif user_ah_score <= MODERATE_AH_UPPER and user_ah_score >= MODERATE_AH_LOWER:
            moderate_user_yom_list.append((user, yom))

        elif user_ah_score <= LOW_AH_UPPER and user_ah_score >= LOW_AH_LOWER:
            low_user_yom_list.append((user, yom))

    return extreme_user_yom_list, moderate_user_yom_list, low_user_yom_list

## Plotting subroutines

In [ ]:
def plot_y1(user_yom_list):
    """
    Plots y1 distribution
    """
    cnt = [0 for i in range(14)]
    for user, yom in user_yom_list:
        y1, y2 = yom
        cnt[y1 - 2008] += 1

    plt.bar([i for i in range(2008, 2022)], cnt)
    plt.xlabel('Year of joining first forum')
    plt.ylabel('#users')
    plt.show()

In [ ]:
def plot_y1_v2(extreme_user_yom_list, moderate_user_yom_list, low_user_yom_list):
    """
    Plots y1 distribution
    """
    cnt_1 = [0 for i in range(14)]
    cnt_2 = [0 for i in range(14)]
    cnt_3 = [0 for i in range(14)]

    labels = [i for i in range(2008, 2022)]

    for user, yom in extreme_user_yom_list:
        y1, y2 = yom
        cnt_1[y1 - 2008] += 1

    for user, yom in moderate_user_yom_list:
        y1, y2 = yom
        cnt_2[y1 - 2008] += 1
    
    for user, yom in low_user_yom_list:
        y1, y2 = yom
        cnt_3[y1 - 2008] += 1

    ticks = np.arange(len(labels))
    width = 0.20

    fig, ax = plt.subplots()
    subplot1 = ax.bar(ticks - width, cnt_1, width, label='Extreme AH', tick_label=labels)
    subplot2 = ax.bar(ticks, cnt_2, width, label='Moderate AH', tick_label=labels)
    subplot3 = ax.bar(ticks + width, cnt_3, width, label='Low AH', tick_label=labels)

    ax.set_ylabel('#users')
    ax.set_xlabel('Year of joining forum 1')
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.legend()
    plt.show()

In [ ]:
def plot_y2(user_yom_list):
    """
    Plots y2 distribution
    """
    cnt = [0 for i in range(14)]
    for user, yom in user_yom_list:
        y1, y2 = yom
        cnt[y2 - 2008] += 1

    plt.bar([i for i in range(2008, 2022)], cnt)
    plt.xlabel('Year of joining second forum')
    plt.ylabel('#users')
    plt.show()

In [ ]:
def plot_y2_v2(extreme_user_yom_list, moderate_user_yom_list, low_user_yom_list):
    """
    Plots y2 distribution
    """
    cnt_1 = [0 for i in range(14)]
    cnt_2 = [0 for i in range(14)]
    cnt_3 = [0 for i in range(14)]

    labels = [i for i in range(2008, 2022)]

    for user, yom in extreme_user_yom_list:
        y1, y2 = yom
        cnt_1[y2 - 2008] += 1

    for user, yom in moderate_user_yom_list:
        y1, y2 = yom
        cnt_2[y2 - 2008] += 1
    
    for user, yom in low_user_yom_list:
        y1, y2 = yom
        cnt_3[y2 - 2008] += 1

    ticks = np.arange(len(labels))
    width = 0.20

    fig, ax = plt.subplots()
    subplot1 = ax.bar(ticks - width, cnt_1, width, label='Extreme AH', tick_label=labels)
    subplot2 = ax.bar(ticks, cnt_2, width, label='Moderate AH', tick_label=labels)
    subplot3 = ax.bar(ticks + width, cnt_3, width, label='Low AH', tick_label=labels)

    ax.set_ylabel('#users')
    ax.set_xlabel('Year of joining forum 2')
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.legend()
    plt.show()

In [ ]:
def plot_delta(user_yom_list):
    """
    Plots (Y2 - Y1) distribution
    """
    max_diff = 0
    for user, yom in user_yom_list:
        y1, y2 = yom
        max_diff = max(max_diff, y2 - y1)

    cnt = [0 for i in range(max_diff + 1)]

    for user, yom in user_yom_list:
        y1, y2 = yom
        cnt[y2 - y1] += 1

    print(cnt)

    plt.bar([i for i in range(1, max_diff + 1)], cnt[1:])
    plt.xlabel('Difference b/w years of joining two forums')
    plt.ylabel('#users')
    plt.show()

In [ ]:
def plot_delta_v2(extreme_user_yom_list, moderate_user_yom_list, low_user_yom_list):
    """
    Plots y2 distribution
    """
    max_diff = 0

    for user, yom in extreme_user_yom_list:
        y1, y2 = yom
        max_diff = max(max_diff, y2 - y1)

    for user, yom in moderate_user_yom_list:
        y1, y2 = yom
        max_diff = max(max_diff, y2 - y1)
    
    for user, yom in low_user_yom_list:
        y1, y2 = yom
        max_diff = max(max_diff, y2 - y1)

    cnt_1 = [0 for i in range(max_diff + 1)]
    cnt_2 = [0 for i in range(max_diff + 1)]
    cnt_3 = [0 for i in range(max_diff + 1)]

    labels = [i for i in range(1, max_diff + 1)]

    for user, yom in extreme_user_yom_list:
        y1, y2 = yom
        cnt_1[y2 - y1] += 1

    for user, yom in moderate_user_yom_list:
        y1, y2 = yom
        cnt_2[y2 - y1] += 1
    
    for user, yom in low_user_yom_list:
        y1, y2 = yom
        cnt_3[y2 - y1] += 1

    ticks = np.arange(len(labels))
    width = 0.20

    fig, ax = plt.subplots()
    subplot1 = ax.bar(ticks - width, cnt_1[1:], width, label='Extreme AH', tick_label=labels)
    subplot2 = ax.bar(ticks, cnt_2[1:], width, label='Moderate AH', tick_label=labels)
    subplot3 = ax.bar(ticks + width, cnt_3[1:], width, label='Low AH', tick_label=labels)

    ax.set_ylabel('#users')
    ax.set_xlabel('Difference b/w years of joining two forums')
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.legend()
    plt.show()

In [ ]:
def plot_wordcloud(user_name, categories, time_1, time_2):
    texts = []

    for category in categories:
        for comment in comments[category]:
            if comment['author'] != user_name:
                continue
            if comment['time'] == 'Not Available':
                continue
            time_ = comment['time'][:10]
            if time_ < time_1 or time_ >= time_2:
                continue
            cleaned_comment_body = ' '.join(clean_text(comment['body']))
            texts.append(cleaned_comment_body)

    texts = ' '.join(texts)
    word_cloud = wordcloud.WordCloud(stopwords=STOP_WORDS,
                                     collocations=False).generate(texts)

    plt.imshow(word_cloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

# plot_wordcloud('excon', ['politics2'], '2018-01-01', '2019-01-01')

## From Poltics U Religion to World News

In [ ]:
time_list = []

for year in range(2008, 2022):
    for month in range(1, 10):
        time_list.append(f'{year}-0{month}-01')
    for month in range(10, 13):
        time_list.append(f'{year}-{month}-01')

In [ ]:
def find_time_idx(time_):
    for i, x in enumerate(time_list):
        if x == time_:
            return i
    return -1

In [ ]:
# Find a set of users who were initially active on Politics, but later migrated
# to Religion.
categories_1 = ['politics2', 'religion']
categories_2 = ['world']

user_yom_list = get_migrated_users(categories_1, categories_2)

In [ ]:
extreme_user_yom_list, moderate_user_yom_list, low_user_yom_list = partition_migrated_users(user_yom_list, categories_1, categories_2)

In [ ]:
print(f'{len(user_yom_list)} | {len(extreme_user_yom_list)} + {len(moderate_user_yom_list)} + {len(low_user_yom_list)}')

### Network study

In [ ]:
reader_addr = '/content/gdrive/MyDrive/DL/CreateDebate/Politics/threads.log'
reader = open(reader_addr, 'rb')
threads = []
try:
    while True:
        e = pickle.load(reader)
        threads.append(e)
except:
    reader.close()

In [ ]:
def build_graph(user_subset, n1 = 0, n2 = 0):
    """Builds support graph and dispute graph from hyper-parameters n1 and n2
    inputs
    :param n1: threshold on number of level-1 comments
    :param n2: threshold on number of direct replies

    output
    (author_map : dict, reverse_map : list, author_count : int, support_graph : nx.DiGraph, support_matrix: list, dispute_graph : nxDiGraph, dispute_matrix : list)
    """

    athr = dict()
    for e in threads:
        if 'root' in e.metaL.keys():
            for key in e.metaL['root'].keys():
                cmnt = e.comments[key]
                cur_athr = cmnt.author
                try:
                    athr[cur_athr] += 1
                except:
                    athr[cur_athr] = 1
        if 'root' in e.metaR.keys():
            for key in e.metaR['root'].keys():
                cmnt = e.comments[key]
                cur_athr = cmnt.author
                try:
                    athr[cur_athr] += 1
                except:
                    athr[cur_athr] = 1

    L1_athr = dict()
    for x in athr:
        if athr[x] >= n1:
            L1_athr[x] = True

    athr = dict()

    def dfs(Map, cmntMap, athr, cid='root'):
        if cid == 'root':
            for key in Map[cid].keys():
                dfs(Map[cid], cmntMap, athr, key)
            return
        cur_author = cmntMap[cid].author

        try:
            athr[cur_author] += len(Map[cid].keys())
        except:
            athr[cur_author] = len(Map[cid].keys())

        for key in Map[cid].keys():
            dfs(Map[cid], cmntMap, athr, key)

    for e in threads:
        if 'root' in e.metaL.keys():
            dfs(e.metaL, e.comments, athr)
        if 'root' in e.metaR.keys():
            dfs(e.metaR, e.comments, athr) 

    A = []
    for x in athr:
        if x not in user_subset:
            continue
        if athr[x] >= n2:
            try:
                z = L1_athr[x]
                A.append(x)
            except KeyError:
                pass

    author_map = dict()
    reverse_map = [""] * len(A)
    author_count = len(A)

    for i in range(author_count):
        author_map[A[i]] = i
        reverse_map[i] = A[i]

    support_matrix = [[0 for j in range(author_count)] for i in range(author_count)]
    dispute_matrix = [[0 for j in range(author_count)] for i in range(author_count)]

    def dfs1(Map, cmntMap, cid='root'):
        if cid == 'root':
            for key in Map[cid].keys():
                dfs1(Map[cid], cmntMap, key)
            return

        cur_author = cmntMap[cid].author
        cur_pol = cmntMap[cid].polarity
        
        if cur_author in author_map and cur_pol != 'Not Available':
            cur_author_id = author_map[cur_author]
            for key in Map[cid].keys():
                nxt_author = cmntMap[key].author
                nxt_pol = cmntMap[key].polarity
                if nxt_author in author_map and nxt_pol != 'Not Available':
                    nxt_author_id = author_map[nxt_author]
                    if cur_pol == nxt_pol:
                        support_matrix[nxt_author_id][cur_author_id] += 1
                    else:
                        dispute_matrix[nxt_author_id][cur_author_id] += 1

        for key in Map[cid].keys():
            dfs1(Map[cid], cmntMap, key)

    for e in threads:
        if 'root' in e.metaL:
            dfs1(e.metaL, e.comments)
        if 'root' in e.metaR:
            dfs1(e.metaR, e.comments)

    support_graph = nx.DiGraph()
    for i in range(author_count):
        for j in range(author_count):
            if support_matrix[i][j] != 0:
                support_graph.add_weighted_edges_from([(i, j, support_matrix[i][j])])

    dispute_graph = nx.DiGraph()
    for i in range(author_count):
        for j in range(author_count):
            if dispute_matrix[i][j] != 0:
                dispute_graph.add_weighted_edges_from([(i, j, dispute_matrix[i][j])])
    
    return (author_map, reverse_map, author_count, support_graph, support_matrix, dispute_graph, dispute_matrix)

In [ ]:
# Vary x_user_yom_list to
#    - user_yom_list
#    - extreme_user_yom_list
#    - moderate_user_yom_list
#    - low_user_yom_list

user_subset = [user for (user, yom) in low_user_yom_list]

_1, _2, _3, support_graph, _4, dispute_graph, _6 = build_graph(user_subset)

try:
    support_graph_r = nx.algorithms.reciprocity(support_graph)
except:
    support_graph_r = 0

try:
    dispute_graph_r = nx.algorithms.reciprocity(dispute_graph)
except:
    dispute_graph_r = 0

avgs, stds = get_stats_from_profile_data(user_subset)

print('Support graph reciprocity', support_graph_r)
print('Dispute graph reciprocity', dispute_graph_r)
print('Average / std-dev of reward points', avgs[0], stds[0])
print('Average / std-dev of efficiency', avgs[1], stds[1])
print('Avergae / std-dev of n_allies', avgs[2], stds[2])
print('Avergae / std-dev of n_enemies', avgs[3], stds[3])
print('Avergae / std-dev of n_hostiles', avgs[4], stds[4])

print(len(user_subset))

### Plots

In [ ]:
plot_y1(user_yom_list)

In [ ]:
plot_y1_v2(extreme_user_yom_list, moderate_user_yom_list, low_user_yom_list)

In [ ]:
plot_y2(user_yom_list)

In [ ]:
plot_y2_v2(extreme_user_yom_list, moderate_user_yom_list, low_user_yom_list)

In [ ]:
plot_delta(user_yom_list)

In [ ]:
plot_delta_v2(extreme_user_yom_list, moderate_user_yom_list, low_user_yom_list)

### Extreme AH group

In [ ]:
for user, yom in extreme_user_yom_list:
    print(f'User: {user}, Politics/Religion Y: {yom[0]}, World Y: {yom[1]}, '
    f'Politics/Religion C: {comment_count["politics2"].get(user, 0) + comment_count["religion"].get(user, 0)}, World C: {comment_count["world"][user]}')

#### User: BurritoLunch

In [ ]:
idx = find_time_idx('2018-01-01')

In [ ]:
while (idx < len(time_list)):
    try:
        plot_wordcloud(user_name='BurritoLunch',
                    categories=['religion', 'politics2'],
                    time_1=time_list[idx],
                    time_2=time_list[idx + 1])
        print(time_list[idx])
    except:
        pass

    finally:
        idx += 1

In [ ]:
idx = find_time_idx('2019-01-01')

In [ ]:
while (idx < len(time_list)):
    try:
        plot_wordcloud(user_name='BurritoLunch',
                    categories=['world'],
                    time_1=time_list[idx],
                    time_2=time_list[idx + 1])
        print(time_list[idx])
    except:
        pass

    finally:
        idx += 1

### Moderate AH group

In [ ]:
for user, yom in moderate_user_yom_list:
    print(f'User: {user}, Politics/Religion Y: {yom[0]}, World Y: {yom[1]}, '
    f'Politics/Religion C: {comment_count["politics2"].get(user, 0) + comment_count["religion"].get(user, 0)}, World C: {comment_count["world"][user]}')

#### User: Cinder000

In [ ]:
idx = find_time_idx('2013-01-01')

while (idx < len(time_list)):
    try:
        plot_wordcloud(user_name='Cinder000',
                    categories=['religion', 'politics2'],
                    time_1=time_list[idx],
                    time_2=time_list[idx + 1])
        print(time_list[idx])
    except:
        pass

    finally:
        idx += 1

In [ ]:
idx = find_time_idx('2013-01-01')

while (idx < len(time_list)):
    try:
        plot_wordcloud(user_name='Cinder000',
                    categories=['world'],
                    time_1=time_list[idx],
                    time_2=time_list[idx + 1])
        print(time_list[idx])
    except:
        pass

    finally:
        idx += 1

### Low AH group

In [ ]:
for user, yom in low_user_yom_list:
    print(f'User: {user}, Politics/Religion Y: {yom[0]}, World Y: {yom[1]}, '
    f'Politics/Religion C: {comment_count["politics2"].get(user, 0) + comment_count["religion"].get(user, 0)}, World C: {comment_count["world"][user]}')

#### User: christjesus

In [ ]:
idx = find_time_idx('2010-01-01')

while (idx < len(time_list)):
    try:
        plot_wordcloud(user_name='christjesus',
                    categories=['religion', 'politics2'],
                    time_1=time_list[idx],
                    time_2=time_list[idx + 1])
        print(time_list[idx])
    except:
        pass

    finally:
        idx += 1

In [ ]:
idx = find_time_idx('2010-01-01')

while (idx < len(time_list)):
    try:
        plot_wordcloud(user_name='christjesus',
                    categories=['world'],
                    time_1=time_list[idx],
                    time_2=time_list[idx + 1])
        print(time_list[idx])
    except:
        pass

    finally:
        idx += 1